In [6]:
from chat_GPT import Chat
import pandas as pd
from tqdm.auto import tqdm

Importing the test data

In [18]:
data = pd.read_csv("~/ownCloud - Michael Saxer (zhaw.ch)@drive.switch.ch/4 Semester/NLP/Projekt2 files/balanced_test.csv")
data.head(100)

,id,comment_text,label
0,c808bb3b0fda7b9e,Fair enough Its not really the freeness or ot...,1
1,9851e54ff65af090,Why did you remove the section I created about...,1
2,b700358de5384110,tommy y dont u suk my fat dik u bitch,1
3,825a091f28fa007d,AND FOR THE LAST FUCKING TIME There is no dis...,1
4,e82e6daeba6eed92,I hate you guys so much,1
...,...,...,...
95,78d6d364412e9c4d,Cant speak for anyone but myself but I will...,1
96,0164123c08c9fcbd,This I hate Eagles for since he made that ...,1
97,1e6e2aa75baef010,What in the HOLY GODS name are you talking abo...,1
98,8b8aac7f0278c237,November UTC Except for the people who a...,0


In [7]:
data = pd.read_csv("data/test/balanced_test.csv")
data.head(10)

,id,comment_text,label
0,c808bb3b0fda7b9e,Fair enough Its not really the freeness or ot...,1
1,9851e54ff65af090,Why did you remove the section I created about...,1
2,b700358de5384110,tommy y dont u suk my fat dik u bitch,1
3,825a091f28fa007d,AND FOR THE LAST FUCKING TIME There is no dis...,1
4,e82e6daeba6eed92,I hate you guys so much,1
5,6d36fa82ca88f214,Hmm I honestly dont know whether the bot will ...,0
6,35b31097a877cf8b,penis im da real nigga in dis hood,1
7,abe1ba4d9c273814,If you deny you are SquealingPig you are a lia...,1
8,14dabf20b28b2991,REDIRECT TalkEastern Suburbs season,0
9,c07accf37e49981a,Jeff G has a vagina and likes to get pounded i...,1


Creating a subset of the data to test the GPT-3 model

In [19]:
messages = data['comment_text'][0:1000].to_list()

In [20]:
data.head(1001)

,id,comment_text,label
0,c808bb3b0fda7b9e,Fair enough Its not really the freeness or ot...,1
1,9851e54ff65af090,Why did you remove the section I created about...,1
2,b700358de5384110,tommy y dont u suk my fat dik u bitch,1
3,825a091f28fa007d,AND FOR THE LAST FUCKING TIME There is no dis...,1
4,e82e6daeba6eed92,I hate you guys so much,1
...,...,...,...
996,11a400cebf1b493b,You got sucked in too I lost my temper with th...,1
997,5887bcfc0e687a16,I also agree and if anyone says that there isn...,0
998,904d277b2e3e8a0f,FILWISE sock You oudated scientif racist age...,0
999,2a3c36a999cd35b8,Why are you such a Bitch Why are you such...,1


## Classify the messages using GPT-3.5 turbo

Importing the chat_GPT class and creating a chat object to classify the messages. We used chunks of 10 messages to reduce the number tokens per request to the API.

# LINUX ONLY!!!

In [15]:
#linux doesent allow access to enviroment variable without explicitly loading it ()
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

In [21]:
responses = []
chat = Chat()

for i in tqdm(range(0, len(messages), 10)):
    chunk = messages[i:i+10]
    responses += chat.batch_create_chats(chunk)

  0%|          | 0/100 [00:00<?, ?it/s]

An error occurred: Error code: 400 - {'error': {'message': "Sorry! We've encountered an issue with repetitive patterns in your prompt. Please try again with a different prompt.", 'type': 'invalid_request_error', 'param': 'prompt', 'code': 'invalid_prompt'}}


Now save the responses to a local csv file.

The process has been interrupted by the api after 800 messages.

In [22]:
#responses
len(responses)

for entry in responses:
    if entry == '-1':
        print("fuck")

fuck
fuck
fuck
fuck
fuck


In [33]:
data = data[0:1000]
data.loc[:, 'gpt_label_zeroshot'] = responses
data['gpt_label_zeroshot'] = data['gpt_label_zeroshot'].astype(int)
#data.to_csv("../data/test/800_test_labeled_gpt.csv", index=False)
data = data[data['gpt_label_zeroshot'] >= 0]
data.to_csv('~/ownCloud - Michael Saxer (zhaw.ch)@drive.switch.ch/4 Semester/NLP/Projekt2 files/sampled_test_data__labeled_result_zeroshot.csv', index=False)


NameError: name 'data_sample' is not defined

Calculate the F1 Score for the GPT-3 labels

In [38]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score


def get_f1_score(data):
  return f1_score(data['label'], data['gpt_label_zeroshot'], average='macro')

In [39]:
get_f1_score(data)


0.5613416234161127

The F1Score is 0.82 which is a relatively good score. Now we'll try to outperform this score using a our BERT transformer model.

## Evaluating fine-tuned distilBERT

Evaluating function

In [11]:
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
from torch.utils.data import TensorDataset, DataLoader
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

def evaluate_model(model_path, test_data_path):
    """
    Evaluates a DistilBERT model for sequence classification using the specified test data path.

    Args:
    model_path (str): Path to the model checkpoint file.
    test_data_path (str): Path to the test data CSV file.
    device (str, optional): Device to evaluate the model on. Defaults to 'cpu'.

    Returns:
    tuple: The macro-averaged accuracy, recall, precision, and F1 score of the model predictions.
    """
    # Load model
    model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    model.to(device)
    model.eval()

    # Load and prepare test data
    df = pd.read_csv(test_data_path)
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    tokenized_data = tokenizer(list(df['comment_text']), padding=True, truncation=True, return_tensors="pt")
    labels = torch.tensor(df['label'].values)
    test_dataset = TensorDataset(tokenized_data['input_ids'], tokenized_data['attention_mask'], labels)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    # Evaluate model
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            inputs, masks, labels = batch
            inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

            outputs = model(inputs, attention_mask=masks)
            _, predicted = torch.max(outputs.logits, dim=1)

            all_preds.extend(predicted.tolist())
            all_labels.extend(labels.tolist())

    # Calculate macro-averaged metrics
    accuracy = accuracy_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    return accuracy, recall, precision, f1

In [12]:
import os
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/My Drive/'
test_data_path = os.path.join(base_path, 'NLP/hate_speech_detection_pipeline/data/test/balanced_test.csv')
model_path = os.path.join(base_path, 'NLP/hate_speech_detection_pipeline/model/model_distil.pth')

# Calculate metrics and output them
accuracy, recall, precision, f1 = evaluate_model(model_path, test_data_path)

print("\n")
print(f'Accuracy: {accuracy}')
print(f'Recall: {recall}')
print(f'Precision: {precision}')
print(f'F1 Score: {f1}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluating:   0%|          | 0/196 [00:00<?, ?it/s]



Accuracy: 0.9009932713873758
Recall: 0.9010059476836778
Precision: 0.9112892766330349
F1 Score: 0.9003717892644056
